In [2]:
import requests, json
import re
from datetime import datetime

In [18]:
apistarturl = 'https://api.weather.gov'

def get_location_url_from_coords(lat,lon):
    return apistarturl + '/points/' + str(lat) + ',' + str(lon)


#HERE IS WHERE YOU CHANGE THE LAT / LON OF THE AREA YOU ARE INTERESTED IN

url = get_location_url_from_coords(33.4483,-112.0740)

#NEW ORLEANS IS USED AS THE DEFAULT 29.9511,-90.0715

print(url)

https://api.weather.gov/points/33.4483,-112.074


In [19]:
r = requests.get(url)
print(r)

<Response [200]>


In [20]:
location_json_obj = json.loads(r.text)
print(location_json_obj.keys())
print(location_json_obj['properties'].keys())

dict_keys(['@context', 'id', 'type', 'geometry', 'properties'])
dict_keys(['@id', '@type', 'cwa', 'forecastOffice', 'gridId', 'gridX', 'gridY', 'forecast', 'forecastHourly', 'forecastGridData', 'observationStations', 'relativeLocation', 'forecastZone', 'county', 'fireWeatherZone', 'timeZone', 'radarStation'])


In [21]:
office = location_json_obj['properties']['gridId']
station = 'K'+ office
print(office)
print(station)
products_in_station_url = apistarturl + '/products/locations/' + office + '/types'
print(products_in_station_url)
r = requests.get(products_in_station_url)
print(r)

PSR
KPSR
https://api.weather.gov/products/locations/PSR/types
<Response [200]>


In [22]:
products_at_station = json.loads(r.text)['@graph']
products_at_station

[{'productCode': 'ABV', 'productName': 'Rawinsonde Data Above 100 Millibars'},
 {'productCode': 'AFD', 'productName': 'Area Forecast Discussion'},
 {'productCode': 'AQA', 'productName': 'Air Quality Alert'},
 {'productCode': 'DSW', 'productName': 'Dust Storm Warning'},
 {'productCode': 'FDI', 'productName': 'Fire Danger Indices'},
 {'productCode': 'FFS', 'productName': 'Flash Flood Statement'},
 {'productCode': 'FFW', 'productName': 'Flash Flood Warning'},
 {'productCode': 'FLS', 'productName': 'Flood Statement'},
 {'productCode': 'FWF',
  'productName': 'Routine Fire Wx Fcst (With/Without 6-10 Day Outlook)'},
 {'productCode': 'FWM', 'productName': 'Miscellaneous Fire Weather Product'},
 {'productCode': 'FWN', 'productName': 'Fire Weather Notification'},
 {'productCode': 'FWO', 'productName': 'Fire Weather Observation'},
 {'productCode': 'FWS', 'productName': 'Suppression Forecast'},
 {'productCode': 'HML', 'productName': 'AHPS XML'},
 {'productCode': 'LSR', 'productName': 'Local Storm

In [23]:
#THIS IS AN IMPORTANT LINE
#v v v v vv v v v v v v v  
desired_product_code = 'PFM'
# ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^
#HERE IS WHERE YOU CHANGE THE PRODUCT CODE TO BE THE THREE LETTER PRODUCT OF INTEREST

for product in products_at_station:
    if product['productCode'] == desired_product_code:
        print(product)
recentProductURL = apistarturl + '/products/types/' + desired_product_code
print(recentProductURL)
r = requests.get(recentProductURL)
print(r)

{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}
https://api.weather.gov/products/types/PFM
<Response [200]>


In [24]:
recentProductURL = apistarturl + '/products/types/' + desired_product_code
recentProductURL

'https://api.weather.gov/products/types/PFM'

In [25]:
recentProductsJSON = json.loads(r.text)
print(recentProductsJSON.keys())
station_products = []
for product in recentProductsJSON['@graph']:
    if product['issuingOffice'] == station:
        station_products.append(product)
station_products[0]

dict_keys(['@context', '@graph'])


{'@id': 'https://api.weather.gov/products/2146c47d-843e-4cb1-bcb9-e6b5392cf923',
 'id': '2146c47d-843e-4cb1-bcb9-e6b5392cf923',
 'wmoCollectiveId': 'FOUS55',
 'issuingOffice': 'KPSR',
 'issuanceTime': '2022-07-19T17:44:00+00:00',
 'productCode': 'PFM',
 'productName': 'Point Forecast Matrices'}

In [26]:
#HERE IS WHERE THE PRODUCTS AVAILABLE ARE REDUCED TO THE MOST RECENT ONE

t1 = datetime.strptime(station_products[0]['issuanceTime'].split('+')[0], '%Y-%m-%dT%H:%M:%S')
t2 = datetime.strptime(station_products[1]['issuanceTime'].split('+')[0], '%Y-%m-%dT%H:%M:%S')
most_recent_date = datetime(1,1,1)
most_recent_product = ''
for product in station_products:
    product_date = datetime.strptime(product['issuanceTime'].split('+')[0], '%Y-%m-%dT%H:%M:%S')
    if most_recent_date < product_date:
        most_recent_product = product
        most_recent_date = product_date

In [27]:
most_recent_product

{'@id': 'https://api.weather.gov/products/2146c47d-843e-4cb1-bcb9-e6b5392cf923',
 'id': '2146c47d-843e-4cb1-bcb9-e6b5392cf923',
 'wmoCollectiveId': 'FOUS55',
 'issuingOffice': 'KPSR',
 'issuanceTime': '2022-07-19T17:44:00+00:00',
 'productCode': 'PFM',
 'productName': 'Point Forecast Matrices'}

In [28]:
most_recent_product_url = most_recent_product['@id']

In [29]:
r =requests.get(most_recent_product_url)
print(r)
product_json = json.loads(r.text)
print(product_json.keys())

<Response [200]>
dict_keys(['@context', '@id', 'id', 'wmoCollectiveId', 'issuingOffice', 'issuanceTime', 'productCode', 'productName', 'productText'])


In [30]:
print(product_json['productText'])


000
FOUS55 KPSR 191744
PFMPSR

Point Forecast Matrices
National Weather Service Phoenix AZ
1044 AM MST Tue Jul 19 2022

AZZ543-192300-
Phoenix-Maricopa AZ
33.43N 112.02W Elev. 1107 ft
1044 AM MST Tue Jul 19 2022

Date                Tue 07/19/22            Wed 07/20/22            Thu 07/21/22
MST 3hrly     02 05 08 11 14 17 20 23 02 05 08 11 14 17 20 23 02 05 08 11 14 17
UTC 3hrly     09 12 15 18 21 00 03 06 09 12 15 18 21 00 03 06 09 12 15 18 21 00

Max/Min                     112          91         111          91         112
Temp                  103108109106101 97 93 93100107109105 99 95 92 93101108110
Dewpt                  58 58 56 55 58 58 59 60 57 56 54 54 57 57 58 56 54 52 50
RH                     23 20 18 19 24 28 32 33 24 19 17 19 25 28 32 29 21 16 14
Wind dir                W  W  W  W  S  W  N  W  W  W  W  W SW SW SW  W  W  W  W
Wind spd                8  9 12 10  1  6  4  8  8 12 14 16 13 11  9  6  8 11 14
Wind gust                       21    18             20 23      

In [31]:
product_json['productText']

'\n000\nFOUS55 KPSR 191744\nPFMPSR\n\nPoint Forecast Matrices\nNational Weather Service Phoenix AZ\n1044 AM MST Tue Jul 19 2022\n\nAZZ543-192300-\nPhoenix-Maricopa AZ\n33.43N 112.02W Elev. 1107 ft\n1044 AM MST Tue Jul 19 2022\n\nDate                Tue 07/19/22            Wed 07/20/22            Thu 07/21/22\nMST 3hrly     02 05 08 11 14 17 20 23 02 05 08 11 14 17 20 23 02 05 08 11 14 17\nUTC 3hrly     09 12 15 18 21 00 03 06 09 12 15 18 21 00 03 06 09 12 15 18 21 00\n\nMax/Min                     112          91         111          91         112\nTemp                  103108109106101 97 93 93100107109105 99 95 92 93101108110\nDewpt                  58 58 56 55 58 58 59 60 57 56 54 54 57 57 58 56 54 52 50\nRH                     23 20 18 19 24 28 32 33 24 19 17 19 25 28 32 29 21 16 14\nWind dir                W  W  W  W  S  W  N  W  W  W  W  W SW SW SW  W  W  W  W\nWind spd                8  9 12 10  1  6  4  8  8 12 14 16 13 11  9  6  8 11 14\nWind gust                       21    1

In [37]:
PFM = product_json['productText']

def returnthree_hr_timeseries_json_from_pfm(PFM):
    
    three_hr_timeseries_json = []
    location_coordinates = re.findall(r'.....N.......W E',PFM)
    location_texts = re.split(r'.....N.......W E', PFM)
    print(location_coordinates)
    
    for index in range(len(location_coordinates)):
        
        time_series_dict = {}
        time_series_dict['location'] = location_coordinates[index]
        
        #date parser
        datesh = re.findall(r'Date         ................................................................', location_texts[1+index])
        date3h = re.findall(r'Date         ...................',datesh[0])
        start_d = date3h[0].replace('Date         ','')
        start_date = start_d.replace('  ','')
        time_series_dict['start_date'] = start_date

        #hour parser
        hours3h = re.findall(r'T 3hrly     .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..',location_texts[1+index])
        hour_csv = hours3h[0].replace('T 3hrly     ','')
        hours = hour_csv.split(' ')
        
        #go through hours and make them additional
        hours_added = []
        daycounter = 0
        lasthour = int(hours[0])
        for hour in hours:
            if int(hour) < lasthour:
                daycounter += 1
            hours_added.append(int(hour)+ daycounter * 24)
            lasthour = int(hour)
        
        #temp parser
        temp = re.findall(r'Temp         ..................................................................', location_texts[1+index])
        forec = temp[0].replace('Temp         ','')
        temps = [forec[i:i+3] for i in range(0, len(forec), 3)]
        
        #dew point parser
        dew = re.findall(r'Dewpt         .................................................................', location_texts[1+index])
        dewp = dew[0].replace('Dewpt         ','')
        dewpoint = [dewp[i:i+3] for i in range(0, len(dewp), 3)]
        
        #wind direction parser
        wind = re.findall(r'Wind dir      .................................................................', location_texts[1+index])
        windDi = wind[0].replace('Wind dir      ','')
        windDirection= [windDi[i:i+3] for i in range(0, len(windDi), 3)]

        #windspeed parser
        winds = re.findall(r'Wind spd      .................................................................',location_texts[1+index])
        windsp = winds[0].replace('Wind spd      ','')
        windSpeed = [windsp[i:i+3] for i in range(0, len(windsp), 3)]

        #clouds parser
        cl= re.findall(r'Clouds        .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. .. ..', location_texts[1+index])
        clo = cl[0].replace('Clouds        ','')
        clouds = [clo[i:i+3] for i in range(0, len(clo), 3)]
        
        #Relative humidity parser
        rh1 = re.findall(r'RH            .................................................................', location_texts[1+index])
        rh2 = rh1[0].replace('RH            ','')
        rh = [rh2[i:i+3] for i in range(0, len(rh2), 3)]
    
        
        #Probability of Precipitation: The next three items are not part of the 3 hour time series
        prec = re.findall(r'PoP 12hr                    ...................................................', location_texts[1+index])
        precip = prec[0].replace('PoP 12hr                     ','')
        precipit = precip.replace('          ',',')
        pop = precipit.split(',')
        time_series_dict['pop12hours'] = pop
        
        #Quantitative Precipitation Forecast
        quantit= re.findall(r'QPF 12hr                 ......      ......      ......      ......      ......',location_texts[1+index])
        quantiprec = quantit[0].replace('QPF 12hr                 ','')
        quantiprecip = quantiprec.replace('       ',',')
        qpf = quantiprecip.split(',')
        time_series_dict['QPF12h'] = qpf
        
        #Not all of the stations measure the probability of snow e.g. Phoenix, Houston. If your PFM doesn't contain this part of
        #the include an if to check the snow
 
        s = re.findall(r'Snow 12hr                .......     .......     .......                       ',location_texts[1+index])
        if len(s) > 0:
            snow = s[0].replace('Snow 12hr                 ','')
            snow1 = snow.replace('                        ','')
            snow12 = snow1.replace('       ',',')
            snow12h = snow12.split(',')
            time_series_dict['Snow12H'] = snow12h
        
        #align hour and temps
        
        ts = []
        
        for index2 in range(len(temps)):
        
        #Hours_added should always append to index2+1 unless the amount of hours and observations is the same.

            ts.append([hours_added[index2],temps[index2],dewpoint[index2], rh[index2], windDirection[index2], windSpeed[index2], clouds[index2]])
        time_series_dict['timeseries'] = ts
        three_hr_timeseries_json.append(time_series_dict)
        
    return three_hr_timeseries_json
 
        
fresh_json = returnthree_hr_timeseries_json_from_pfm(PFM)
print(fresh_json[0:20])
    

['33.43N 112.02W E', '32.66N 114.61W E', '33.39N 110.79W E', '33.61N 111.91W E', '33.69N 112.08W E', '33.59N 112.37W E', '33.30N 111.67W E', '32.95N 112.71W E', '32.89N 112.72W E', '32.96N 111.77W E', '33.62N 114.72W E', '32.85N 115.57W E', '32.83N 115.68W E']
[{'location': '33.43N 112.02W E', 'start_date': ' Tue 07/19/22', 'pop12hours': [' 0', ' 5', ' 5', ' 5', ' 5'], 'QPF12h': ['     0', '    0', '    0', '    0', '    0'], 'timeseries': [[2, '   ', '   ', '   ', '   ', '   ', '   '], [5, '   ', '   ', '   ', '   ', '   ', '   '], [8, '   ', '   ', '   ', '   ', '   ', '   '], [11, '103', '58 ', '23 ', ' W ', ' 8 ', 'B1 '], [14, '108', '58 ', '20 ', ' W ', ' 9 ', 'FW '], [17, '109', '56 ', '18 ', ' W ', '12 ', 'SC '], [20, '106', '55 ', '19 ', ' W ', '10 ', 'SC '], [23, '101', '58 ', '24 ', ' S ', ' 1 ', 'B1 '], [26, ' 97', '58 ', '28 ', ' W ', ' 6 ', 'B1 '], [29, ' 93', '59 ', '32 ', ' N ', ' 4 ', 'B1 '], [32, ' 93', '60 ', '33 ', ' W ', ' 8 ', 'SC '], [35, '100', '57 ', '24 ', ' W 

In [38]:
# the time series contains the temperature, dew point, relative humidity, wind direction, wind speed, and clouds
fresh_json

[{'location': '33.43N 112.02W E',
  'start_date': ' Tue 07/19/22',
  'pop12hours': [' 0', ' 5', ' 5', ' 5', ' 5'],
  'QPF12h': ['     0', '    0', '    0', '    0', '    0'],
  'timeseries': [[2, '   ', '   ', '   ', '   ', '   ', '   '],
   [5, '   ', '   ', '   ', '   ', '   ', '   '],
   [8, '   ', '   ', '   ', '   ', '   ', '   '],
   [11, '103', '58 ', '23 ', ' W ', ' 8 ', 'B1 '],
   [14, '108', '58 ', '20 ', ' W ', ' 9 ', 'FW '],
   [17, '109', '56 ', '18 ', ' W ', '12 ', 'SC '],
   [20, '106', '55 ', '19 ', ' W ', '10 ', 'SC '],
   [23, '101', '58 ', '24 ', ' S ', ' 1 ', 'B1 '],
   [26, ' 97', '58 ', '28 ', ' W ', ' 6 ', 'B1 '],
   [29, ' 93', '59 ', '32 ', ' N ', ' 4 ', 'B1 '],
   [32, ' 93', '60 ', '33 ', ' W ', ' 8 ', 'SC '],
   [35, '100', '57 ', '24 ', ' W ', ' 8 ', 'SC '],
   [38, '107', '56 ', '19 ', ' W ', '12 ', 'FW '],
   [41, '109', '54 ', '17 ', ' W ', '14 ', 'FW '],
   [44, '105', '54 ', '19 ', ' W ', '16 ', 'SC '],
   [47, ' 99', '57 ', '25 ', 'SW ', '13 ', 'B1 ']